In [ ]:
# !pip install wandb --upgrade

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import libraries 
import pandas as pd 
import numpy as np 
import scipy.stats
from functools import reduce

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

from tqdm import trange

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import random
import math

from torch.utils.tensorboard import SummaryWriter
import torch.optim as optim
import matplotlib.pyplot as plt
import wandb

In [ ]:
# CODICE DA METTERE PRIMA DELLA NET


# subtype mapping
# 0 -> healthy
# 1 -> tumor
dict_label = {'TCGA-LUAD0' : 0,
              'TCGA-LUAD1' : 1,
              'TCGA-LUSC0' : 2,
              'TCGA-LUSC1' : 3}

# dict_label = {'TCGA-LUAD0' : 0,
#               'TCGA-LUAD1' : 1,
#               'TCGA-LUSC0' : 2,
#               'TCGA-LUSC1' : 3,
#               'TCGA-CPTAC0' : 4,
#               'TCGA-CPTAC1' : 5,
#               'TCGA-BRCA0' : 6,
#               'TCGA-BRCA1' : 7,
#               'TCGA-KIRC0' : 8,
#               'TCGA-KIRC1' : 9
#               }

# input files
file_meth = "/content/drive/MyDrive/University/Bioinformatics/dataset/meth_feat_sel_dataset.csv"
file_mRNA = "/content/drive/MyDrive/University/Bioinformatics/dataset/mRNA_feat_sel_dataset.csv"
file_miRNA = "/content/drive/MyDrive/University/Bioinformatics/dataset/miRNA_feat_sel_dataset.csv"

# leggo in un DF i 3 csv feat_sel
df_meth = pd.read_csv(file_meth)
df_mRNA = pd.read_csv(file_mRNA)
df_miRNA = pd.read_csv(file_miRNA)

# number of features (da passare alla rete)
# -2 perchè non consideriamo il case_id e il subtype
meth_dim = df_meth.shape[1] - 2
mRNA_dim = df_mRNA.shape[1] - 2
miRNA_dim = df_miRNA.shape[1] - 2 

# join su case_id 
dataset = reduce(lambda left,right: pd.merge(left,right,on=['case_id','subtype']), [df_miRNA,df_mRNA,df_meth] )
#dataset = reduce(lambda left,right: pd.merge(left,right,on=['case_id']), [df_miRNA,df_mRNA,df_meth] )

# just a check
print(dataset.shape)
print(meth_dim + miRNA_dim + mRNA_dim + 2)

# train_test_split stratified su colonna subtype
# we are done!

(1121, 6741)
6741


In [ ]:
# 1. Start a new run
wandb.init(project='bioinfo', entity='latte')


# # 2. Save model inputs and hyperparameters
# config = wandb.config

In [ ]:
def seed_everything(seed): 
    '''
    seed_everything(seed)
        Ensure deterministic behaviour. Use a seed that will be used to seed pytroch, numpy and python right at the start of main process.

    Parameters
    ----------
    seed : int
        the seed
    
    See also: 
    ----------
        ref: https://discuss.pytorch.org/t/reproducibility-with-all-the-bells-and-whistles/81097
        ref: https://pytorch.org/docs/stable/notes/randomness.html
    '''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    # if using 'cuda'
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.enabled=False
    torch.backends.cudnn.deterministic=True

# set random seed
SEED = 769
seed_everything(SEED)

In [ ]:
# other params
TRAIN_CLASSIFIER=False
NORMALIZATION=False

In [ ]:
# Net (only classifier)
class LambdaLayer(nn.Module):
    def __init__(self, lambd):
        super(LambdaLayer, self).__init__()
        self.lambd = lambd
    def forward(self, x):
        return self.lambd(x)

class Encoder (nn.Module):
    def __init__(self,meth_dim,mRNA_dim,miRNA_dim):
        super().__init__()
        self.meth_dim,self.mRNA_dim,self.miRNA_dim = meth_dim,mRNA_dim,miRNA_dim
        self.fc1_meth = nn.Linear(meth_dim, 25)
        self.m_meth = nn.BatchNorm1d(25)

        self.fc1_mRNA = nn.Linear(mRNA_dim, 25)
        self.m_mRNA = nn.BatchNorm1d(25)

        self.fc1_miRNA = nn.Linear(miRNA_dim, 25)
        self.m_miRNA = nn.BatchNorm1d(25)

        self.fc2 = nn.Linear(75, 75)
        self.m = nn.BatchNorm1d(75)
        self.fcm = nn.Linear(75, 75)
        self.fcstd = nn.Linear(75, 75)
        self.gl = nn.GELU()

    def forward(self, x): 
        def sampling(args):
            z_mean, z_log_var = args
            return z_mean + torch.exp(0.5 * z_log_var) * torch.normal(z_mean.size(), seed=0)
        #recupero dati input
        x_m = torch.tensor(x[:,:self.meth_dim])
        x_R = torch.tensor(x[:,self.meth_dim:self.meth_dim+self.mRNA_dim])
        x_p = torch.tensor(x[:,self.meth_dim+self.mRNA_dim:self.meth_dim+self.mRNA_dim+self.miRNA_dim])

        #independent layers for each omic
        x_meth = self.gl(self.m_meth(self.fc1_meth(x_m)))
        x_mRNA = self.gl(self.m_mRNA(self.fc1_mRNA(x_R)))
        x_prot = self.gl(self.m_miRNA(self.fc1_miRNA(x_p)))
        
        #concatenazione dei vari output da processare fino allo shared layer
        x = torch.cat([x_meth,x_mRNA,x_prot],axis = 1 )
        x = self.gl(self.m(self.fc2(x)))
        mean = self.fcm(x)
        std = self.fcstd(x)
        #z = LambdaLayer(sampling, [mean, var])
        return x

### GAN
# Discriminator - D
class Decoder(nn.Module):
    def __init__(self,meth_dim,mRNA_dim,miRNA_dim):
        super().__init__()
        self.dec = nn.Sequential(
                      nn.Linear(75, 75),
                      nn.GELU(),
                      nn.Linear(75,meth_dim+mRNA_dim+miRNA_dim)          
                      )
        
    def forward(self, x):
        return self.dec(x)

class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.disc = nn.Sequential(
                nn.Linear(75, 75),
                nn.GELU(),
                nn.Linear(75,1),
                nn.Sigmoid()) #because we will normalize inputs. 
    def forward(self, x):
        return self.disc(x)

        # Generator - G
class Discriminator_CL(nn.Module):
    def __init__(self):
        super().__init__()
        self.disc = nn.Sequential(
                nn.Linear(75, 75),
                nn.GELU(),
                nn.Linear(75,5),
                nn.Sigmoid() )
    def forward(self, x):
        return self.disc(x)

In [ ]:
X = np.array(dataset.drop(['subtype','case_id'],axis=1).values)
Y = dataset.subtype.values

In [ ]:
def toone(x):
  arr = np.zeros(5)
  arr[int(x)]=1
  return arr

In [ ]:
# split dataset (non serve per la GAN)
X_train, X_test, Y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=SEED, stratify = Y)

In [ ]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

In [ ]:
parameters_dict = {
    'optimizer': {
        'values': ['adam'] #, 'sgd']
        },
    'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 1e-5,
        'max': 1e-3
      },
    'batch_size': {
        'values': [32,64]
      }
    }
sweep_config['parameters'] = parameters_dict

In [ ]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'accuracy'},
 'parameters': {'batch_size': {'values': [32, 64]},
                'learning_rate': {'distribution': 'uniform',
                                  'max': 0.001,
                                  'min': 1e-05},
                'optimizer': {'values': ['adam']}}}


In [ ]:
sweep_id = wandb.sweep(sweep_config, project="bioinfo")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: fdq4fe7f
Sweep URL: https://wandb.ai/latte/bioinfo/sweeps/fdq4fe7f


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
      # If called by wandb.agent, as below,
      # this config will be set by Sweep Controller
      config = wandb.config

      # TRAINING 8=====D  
      encoder = Encoder(meth_dim,mRNA_dim,miRNA_dim).to(device)
      decoder = Decoder(meth_dim,mRNA_dim,miRNA_dim).to(device)
      discriminator = Discriminator().to(device)
      discriminator_cl = Discriminator_CL().to(device)
      #noise = torch.randn((config.batch_size,in_dimension)).to(device)

      #transforms = transforms.Compose(
      #    [transforms.ToTensor() ,transforms.Normalize((mean,),(stdev,))])

      # optimizer
      if config.optimizer == "sgd":
        optim_disc = optim.SGD(discriminator.parameters(),lr = config.learning_rate,momentum=0.9)
        optim_disc_cl = optim.SGD(discriminator_cl.parameters(),lr = config.learning_rate,momentum=0.9)
        optim_gen = optim.SGD(encoder.parameters(),lr = config.learning_rate,momentum=0.9)
        optim_dec = optim.SGD(decoder.parameters(),lr = config.learning_rate,momentum=0.9)
      elif config.optimizer == "adam":
        optim_disc = optim.Adam(discriminator.parameters(),lr = config.learning_rate)
        optim_disc_cl = optim.Adam(discriminator_cl.parameters(),lr = config.learning_rate)
        optim_gen = optim.Adam(encoder.parameters(),lr = config.learning_rate)
        optim_dec = optim.Adam(decoder.parameters(),lr = config.learning_rate)

      bce = nn.BCELoss()
      mse = nn.MSELoss()
      cle = nn.CrossEntropyLoss()

      # SummaryWriter -> log tensorboard
      # writer_fake = SummaryWriter(f"runs/GAN_bio/fake")
      # writer_real = SummaryWriter(f"runs/GAN_bio/real")

      step = 0
      num_epochs = 50
      epochs = trange(num_epochs)
      lossesD, lossesCL, lossesG, lossesDec = [],[],[],[]
      accuracies_test = []

      # TRAINING GAN
      for epoch in epochs:
          # batch
          permutation = torch.randperm(X_train.shape[0])
          lossD, lossG = 0,0

          # compute metrics curr_epoch -> mean over all batches
          lossD_curr_epoch = list()
          lossE_curr_epoch = list()
          lossDec_curr_epoch = list()
          loss_discl_curr_epoch = list()
          accuracy_curr_epoch = list()

          # train a Batch (dim BS)
          for i in range(0,X_train.shape[0], config.batch_size):
              
              # extract X,Y (dimension config.batch_size)
              indices = permutation[i:i+config.batch_size]
              batch_x, batch_y = torch.FloatTensor(X_train)[indices], torch.LongTensor(Y_train)[indices]
              # generate fake img

              latent_real = torch.Tensor(np.random.normal(size=(batch_x.shape[0], 75)))

              fake = encoder(batch_x)
              
              # train discriminator max - LOG(D(real)) + log(1-D(G(z))) opt when D(real)==1 and D(G(z))==0
              # real img
              disc_real = discriminator(latent_real).view(-1)
              lossD_real = bce(disc_real,torch.ones_like(disc_real)) #max LOG(D(real)) part of the loss
              # fake img
              disc_fake = discriminator(fake).view(-1)  
              lossD_fake = bce(disc_fake,torch.zeros_like(disc_real))
              # final disc loss
              lossD = (lossD_real + lossD_fake) / 2
              discriminator.zero_grad()
              #lossD.backward(retain_graph=True) 
              
              # train generator min log (1-D(G(z))) opt when D(G(z))==1 ----> max log(D(G(z)))
              output = discriminator(fake).view(-1)
              # we use ones because of the structure of BCELoss ylog(D(G(z))) + (1-y)log(1-D(G(z)))
              lossE = bce(output, torch.ones_like(output)) 
              encoder.zero_grad()
            # lossE.backward()

              # reconstruction loss
              decoded = decoder(fake)
              lossDec = mse(decoded,batch_x)
              decoder.zero_grad()

              #classification loss 
              result = discriminator_cl(fake)
              loss_discl = cle(result,batch_y)
              discriminator_cl.zero_grad()


              lossf = lossD+lossE+lossDec+100000*loss_discl
              lossf.backward()
              optim_dec.step()
              optim_gen.step()
              optim_disc.step()
              optim_disc_cl.step()
              winners = result.argmax(dim=1)
              corrects = (winners == batch_y)
              accuracy = corrects.sum().float() / float( batch_y.size(0) )

              lat_test = encoder(torch.FloatTensor(X_test))
              win_test = discriminator_cl(lat_test)
              
              winners_test = win_test.argmax(dim=1)
              corrects_test = (winners_test == torch.Tensor(y_test))
              accuracy_test = corrects_test.sum().float() / float( torch.Tensor(y_test).size(0) )


              # update metrics curr_epoch
              lossD_curr_epoch.append(lossD.item())
              lossE_curr_epoch.append(lossE.item())
              lossDec_curr_epoch.append(lossDec.item())
              loss_discl_curr_epoch.append(loss_discl.item())
              accuracy_curr_epoch.append(accuracy_test.item())

              # lossD --> GAN fake/norm
              # lossE --> GAN
              # lossDec --> reconstruction loss (autoenc)
              # lossCL --> classificatore finale
              epochs.set_description("lossD %.2f lossE %.2f lossDec %.2f lossCL %.2f accuracy %.2f" %(lossD,lossE,lossDec,loss_discl,accuracy_test))
          
          # logga tutto -> metrics curr_epoch
          wandb.log({"lossD": np.mean(np.array(lossD_curr_epoch)),
                    "lossE": np.mean(np.array(lossE_curr_epoch)),
                    "lossDec": np.mean(np.array(lossDec_curr_epoch)),
                    "loss_discl": np.mean(np.array(loss_discl_curr_epoch)),
                    "accuracy": np.mean(np.array(accuracy_curr_epoch)) })
          
          # append loss computed on the last batch_size
          lossesD.append(lossD)
          lossesG.append(lossG)  
          lossesDec.append(lossDec)
          accuracies_test.append(accuracy_test)


In [ ]:
# Save the model in the exchangeable ONNX format
#for model,name in zip([encoder, decoder, discriminator, discriminator_cl],['encoder','decoder','disciminator','discriminator_clf']):

# filename = f"encoder.onnx"
# torch.onnx.export(encoder, batch_x, filename)
# wandb.save(filename)

In [ ]:
wandb.agent(sweep_id, train, count=10)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: eua6as7o with config:
wandb: 	batch_size: 32
wandb: 	learning_rate: 6.133991228763558e-05
wandb: 	optimizer: adam


  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
lossD 0.70 lossE 0.42 lossDec 0.45 lossCL 0.92 accuracy 0.98: 100%|██████████| 50/50 [00:50<00:00,  1.01s/it]


lossD,0.70121
lossE,0.40933
lossDec,0.53516
loss_discl,0.92093
accuracy,0.97995
_runtime,54
_timestamp,1630766813
_step,49


lossD,▁▂▃▃▅▆▆▇▇▇███████▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▄▃▃▃▂▂▂▂
lossE,█▇▆▆▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lossDec,███▇▇▆▅▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_discl,█▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▅▆▇████████████████████████████████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: Agent Starting Run: drt81fsv with config:
wandb: 	batch_size: 32
wandb: 	learning_rate: 0.00043102185465157625
wandb: 	optimizer: adam
wandb: Currently logged in as: latte (use `wandb login --relogin` to force relogin)


...

lossD,▅██▇▅▅▄▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lossE,█▂▁▂▂▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁
lossDec,█▆▄▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_discl,█▆▄▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▅▆▆▆▆▆▆▆▆▇▇▇██▇▇▇▇██████████▇▇▇▇▇▇█▇██▇
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇▇██
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


wandb: Agent Starting Run: mgj6kwrk with config:
wandb: 	batch_size: 64
wandb: 	learning_rate: 0.0004313933415651649
wandb: 	optimizer: adam


lossD 0.56 lossE 0.41 lossDec 0.50 lossCL 0.91 accuracy 0.99: 100%|██████████| 50/50 [00:28<00:00,  1.74it/s]


lossD,0.5631
lossE,0.40591
lossDec,0.47016
loss_discl,0.90755
accuracy,0.98806
_runtime,34
_timestamp,1630767313
_step,49


lossD,▆▇██▇▇▇▆▅▅▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁
lossE,█▄▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lossDec,█▇▆▅▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_discl,█▇▆▅▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,▁▅▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████████████████████
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███


In [ ]:
### GAN
config = wandb.config

# BEST RUN HYPERPARAMS
hyperparameters = {
              'epochs' : 100,
              'batch_size': 32,
              'learning_rate': 0.000431,
              'optimizer': 'adam'}
              
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# tell wandb to get started
with wandb.init(project="bioinfo", config=hyperparameters):
  # access all HPs through wandb.config, so logging matches execution!
  config = wandb.config

  encoder = Encoder(meth_dim,mRNA_dim,miRNA_dim).to(device)
  decoder = Decoder(meth_dim,mRNA_dim,miRNA_dim).to(device)
  discriminator = Discriminator().to(device)
  discriminator_cl = Discriminator_CL().to(device)
  #noise = torch.randn((config.batch_size,in_dimension)).to(device)

  #transforms = transforms.Compose(
  #    [transforms.ToTensor() ,transforms.Normalize((mean,),(stdev,))])

  # optimizer
  if config.optimizer == "sgd":
    optim_disc = optim.SGD(discriminator.parameters(),lr = config.learning_rate,momentum=0.9)
    optim_disc_cl = optim.SGD(discriminator_cl.parameters(),lr = config.learning_rate,momentum=0.9)
    optim_gen = optim.SGD(encoder.parameters(),lr = config.learning_rate,momentum=0.9)
    optim_dec = optim.SGD(decoder.parameters(),lr = config.learning_rate,momentum=0.9)
  elif config.optimizer == "adam":
    optim_disc = optim.Adam(discriminator.parameters(),lr = config.learning_rate)
    optim_disc_cl = optim.Adam(discriminator_cl.parameters(),lr = config.learning_rate)
    optim_gen = optim.Adam(encoder.parameters(),lr = config.learning_rate)
    optim_dec = optim.Adam(decoder.parameters(),lr = config.learning_rate)

  bce = nn.BCELoss()
  mse = nn.MSELoss()
  cle = nn.CrossEntropyLoss()

  # SummaryWriter -> log tensorboard
  # writer_fake = SummaryWriter(f"runs/GAN_bio/fake")
  # writer_real = SummaryWriter(f"runs/GAN_bio/real")

  step = 0
  epochs = trange(config.epochs)
  lossesD, lossesCL, lossesG, lossesDec = [],[],[],[]
  accuracies_test = []

  # TRAINING GAN
  for epoch in epochs:
      # batch
      permutation = torch.randperm(X_train.shape[0])
      lossD, lossG = 0,0

      # compute metrics curr_epoch -> mean over all batches
      lossD_curr_epoch = list()
      lossE_curr_epoch = list()
      lossDec_curr_epoch = list()
      loss_discl_curr_epoch = list()
      accuracy_curr_epoch = list()

      # train a Batch (dim BS)
      for i in range(0,X_train.shape[0], config.batch_size):
          
          # extract X,Y (dimension config.batch_size)
          indices = permutation[i:i+config.batch_size]
          batch_x, batch_y = torch.FloatTensor(X_train)[indices], torch.LongTensor(Y_train)[indices]
          # generate fake img

          latent_real = torch.Tensor(np.random.normal(size=(batch_x.shape[0], 75)))

          fake = encoder(batch_x)
          
          # train discriminator max - LOG(D(real)) + log(1-D(G(z))) opt when D(real)==1 and D(G(z))==0
          # real img
          disc_real = discriminator(latent_real).view(-1)
          lossD_real = bce(disc_real,torch.ones_like(disc_real)) #max LOG(D(real)) part of the loss
          # fake img
          disc_fake = discriminator(fake).view(-1)  
          lossD_fake = bce(disc_fake,torch.zeros_like(disc_real))
          # final disc loss
          lossD = (lossD_real + lossD_fake) / 2
          discriminator.zero_grad()
          #lossD.backward(retain_graph=True) 
          
          # train generator min log (1-D(G(z))) opt when D(G(z))==1 ----> max log(D(G(z)))
          output = discriminator(fake).view(-1)
          # we use ones because of the structure of BCELoss ylog(D(G(z))) + (1-y)log(1-D(G(z)))
          lossE = bce(output, torch.ones_like(output)) 
          encoder.zero_grad()
        # lossE.backward()

          # reconstruction loss
          decoded = decoder(fake)
          lossDec = mse(decoded,batch_x)
          decoder.zero_grad()

          #classification loss 
          result = discriminator_cl(fake)
          loss_discl = cle(result,batch_y)
          discriminator_cl.zero_grad()


          lossf = loss_discl+lossDec #lossD+lossE+lossDec+100000*loss_discl
          lossf.backward()
          optim_dec.step()
          # optim_gen.step()
          # optim_disc.step()
          optim_disc_cl.step()
          winners = result.argmax(dim=1)
          corrects = (winners == batch_y)
          accuracy = corrects.sum().float() / float( batch_y.size(0) )

          lat_test = encoder(torch.FloatTensor(X_test))
          win_test = discriminator_cl(lat_test)
          
          winners_test = win_test.argmax(dim=1)
          corrects_test = (winners_test == torch.Tensor(y_test))
          accuracy_test = corrects_test.sum().float() / float( torch.Tensor(y_test).size(0) )


          # update metrics curr_epoch
          lossD_curr_epoch.append(lossD.item())
          lossE_curr_epoch.append(lossE.item())
          lossDec_curr_epoch.append(lossDec.item())
          loss_discl_curr_epoch.append(loss_discl.item())
          accuracy_curr_epoch.append(accuracy_test.item())

          # lossD --> GAN fake/norm
          # lossE --> GAN
          # lossDec --> reconstruction loss (autoenc)
          # lossCL --> classificatore finale
          epochs.set_description("lossD %.2f lossE %.2f lossDec %.2f lossCL %.2f accuracy %.2f" %(lossD,lossE,lossDec,loss_discl,accuracy_test))
      
      # logga tutto -> metrics curr_epoch
      wandb.log({"lossD": np.mean(np.array(lossD_curr_epoch)),
                 "lossE": np.mean(np.array(lossE_curr_epoch)),
                 "lossDec": np.mean(np.array(lossDec_curr_epoch)),
                 "loss_discl": np.mean(np.array(loss_discl_curr_epoch)),
                 "accuracy": np.mean(np.array(accuracy_curr_epoch)) })
      
      # append loss computed on the last batch_size
      lossesD.append(lossD)
      lossesG.append(lossG)  
      lossesDec.append(lossDec)
      accuracies_test.append(accuracy_test)